In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet, stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
tqdm.pandas()

In [ ]:
stopword_list = stopwords.words("english")

# Map POS tags to WordNet tags
def get_wordnet_pos(pos_tag):
    tag = pos_tag[0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def preprocess_text(text, stem=False, lemmatize=True, stop_words=stopword_list):
    # Convert to lowercase and remove punctuations and characters, then strip
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    # Tokenize (convert from string to list)
    words = text.split()    
    
    # Remove stopwords
    if stop_words:
        words = [word for word in words if word not in stop_words]
    
    # Lemmatization (convert the word into root word)
    if lemmatize:
        lemmatizer = nltk.stem.WordNetLemmatizer()
        pos_tags = nltk.pos_tag(words)
        words = [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in pos_tags]

    # Stemming (remove -ing, -ly, ...)
    if stem:
        stemmer = nltk.stem.PorterStemmer()
        words = [stemmer.stem(word) for word in words]

    # Convert back to string from list
    text = " ".join(words)
    
    return text


In [ ]:
df = pd.read_csv("data/data_complete.csv")
df['text_clean'] =  df['text'].progress_apply(lambda x: preprocess_text(x))

In [ ]:
#Currently no lemmatization or stemming
X = train['text_clean']
y = train['sdg']
# X_test = 
# y_test = 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
# vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X_vec = vectorizer.fit_transform(X)
X_vec = np.asarray(X_vec.todense()) # convert sparse matrix into dense matrix
# X_test_vec = vectorizer.fit_transform(X_test)
# X_test_vec = np.asarray(X_test_vec.todense()) # convert sparse matrix into dense matrix

In [ ]:
# Apply TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer() #by default applies "l2" normalisation
X_tfidf = tfidf.fit_transform(X_vec)
X_tfidf = np.asarray(X_tfidf.todense())

# X_test_tfidf = tfidf.fit_transform(X_test_vec)
# X_test_tfidf = np.asarray(X_test_tfidf.todense())

In [ ]:
# split datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.1, random_state = 0)
# X_train = X_train_tfidf
# X_test = X_test_tfidf

In [ ]:
# training NB Classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
# Predicting the test set results
y_pred = clf.predict(X_test)
# Confusion Matrix
from sklearn.metrics import confusion_matrix, f1_score



In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Accuracy: ", round(100*cm.diagonal().sum()/cm.sum(), 3), '%' )

In [ ]:
f1_score(y_test, y_pred, average="weighted")

In [ ]:
# Train SVM Classifier
from sklearn.svm import SVC
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)
y_pred2 = classifier.predict(X_test)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred2)
print("Accuracy: ", round(100*cm.diagonal().sum()/cm.sum(), 3), '%' )
cm

In [ ]:
f1_score(y_test, y_pred2, average="weighted")